# Оценки качества моделей

## Оценка качества классификации

### Бинарная классификация

Пусть мы решаем задачу бинарной классификации, т.е. $y=\{0,1\}$; алгоритм предсказывает принадлежность каждого объекта одному из классов. Возможны следующие ситуации: 
* TP — true positive
* TN — true negative
* FP — false positive
* FN — false negative

Введем **матрицу ошибок (confusion matrix)**:

<img src="pictures/cm.png" width=230 height=230 />

**Пример:** пусть у нас есть модель определения какого-то заболевания, которая относит к классу 1 носителя заболевания, а к 0 - здорового человека. В этом случае классификатор может совершить 2 типа ошибок:
* False Positive (FP или ошибка первого рода) - признать здорового человека больным;
* False Negative (FP или ошибка второго рода) - признать больного человека здоровым.

Для разных задач ошибки первого и второго рода могут иметь разную важность и разный вес, например, для нашего классификатора гораздо хуже ошибки второго рода, и за их появлением нужно следить тщательнее.

#### Метрики качества

**Аккуратность (accuracy)** — доля правильных ответов: количество верно предсказанных ответов / количество всех примеров: $\frac{TP + TN}{TP + TN + FP + FN}$

**Точность (precision)** — доля объектов, названных классификатором положительными и при этом действительно являющимися положительными: $\frac{TP}{TP + FP} = \frac{\text{True positives}}{\text{Predicted positive}}$

**Полнота (recall)** — доля объектов положительного класса из всех объектов положительного класса, названных классификатором: $\frac{TP}{TP + FN} = \frac{\text{True positives}}{\text{Positives}}$

<img src="pictures/prec_rec.png" width=350 height=350 />

**F-Score** — среднее гармоническое между полнотой (Recall) и точностью (Precision):
$$F_\beta = \frac{(\beta^2 + 1) Precision Recall}{\beta^2 Precision + Recall}$$
или при $\beta = 1$
$$F_1 = \frac{2 Precision Recall}{Precision + Recall}$$

**Specificity** - доля верных срабатываний классификатора к общему числу объектов за пределами класса. Иначе говоря то, насколько часто классификатор правильно не относит объекты к классу.
$Spc=\frac{TN}{FP+TN}$

**Площадь под кривой рабочих характеристик (ROC = Receiver Operating Characteristics, AUC = Area Under the Curve)** 
* ROC-кривая (receiver operating characteristic, рабочая характеристика приёмника) — кривая, позволяющая визуализировать результат работы бинарного классификатора на фиксированной тестовой выборке. Она показывает долю истинно ложно положительных примеров (false positive rate) в сравнении с долей истинно положительных примеров (true positive rate):
    * **FPR (false positive rate)**: $\frac{FP}{FP+TN} = \frac{\text{False positives}}{\text{Negatives}}$
    * **TPR (true positive rate)**: $\frac{TP}{TP + FN} = \frac{\text{True positives}}{\text{Positives}} = Recall$
* AUC (area under the curve) - площадь под этой кривой, которую используют как оценку качества алгоритма.

<img src="pictures/roc-curve.png" width=400 height=400 />

[Алгоритм построения ROC-кривой (из блога Дьяконова)](https://dyakonov.org/2017/07/28/auc-roc-%D0%BF%D0%BB%D0%BE%D1%89%D0%B0%D0%B4%D1%8C-%D0%BF%D0%BE%D0%B4-%D0%BA%D1%80%D0%B8%D0%B2%D0%BE%D0%B9-%D0%BE%D1%88%D0%B8%D0%B1%D0%BE%D0%BA/):
1. Запустить классификатор на некоторй выборке (Табл. 1)
2. Отсортировать результаты по уверенности классификатора в принадлежности объекта к классу (Табл. 2). 
    * В идеале её столбец с классом будет упорядочен (сначала идут 1, потом 0);
    * В наихудшем случае порядок будет обратный (сначала 0, потом 1);
    * В случае «слепого угадывания» будет случайное распределение 0 и 1.
3. Возьмем единичный квадрат и разобьем его на m равных частей горизонтальными линиями и на n – вертикальными, где m – число 1 среди правильных меток теста (в нашем примере m=3), n – число нулей (n=4). В результате квадрат разбивается сеткой на m×n блоков. 4. Пока не кончились элементы будем просматривать строки табл. 2 сверху вниз и прорисовывать на сетке линии, переходя их одного узла в другой из точки (0, 0) в точку (1, 1) (сделаем в сумме m шагов вверх и n шагов вправо):
    1. Берем объект с максимальной уверенностью
    2. Сравнить метку с реальной
    3. Пересчитать TPR и FPR на взятых объектах
    4. Поставить точку, если обе характеристики не NaN / $\pm \infty$

Сетка на рисунке разбила квадрат на m×n блоков. Ровно столько же пар вида (объект класса 1, объект класса 0), составленных из объектов тестовой выборки. Каждый закрашенный блок на рис. 1 соответствует паре (объект класса 1, объект класса 0), для которой алгоритм правильно предсказал порядок (объект класса 1 получил оценку выше, чем объект класса 0), незакрашенный блок – паре, на которой ошибся.

Таким образом, AUC ROC равен доле пар объектов вида (объект класса 1, объект класса 0), которые алгоритм верно упорядочил, т.е. первый объект идёт в упорядоченном списке раньше. Число точек не превосходит число объектов.

<img src="pictures/rocauc1.png" width=800 height=800 />

* Если ROC-AUC < 0.5, то нужно поменять метки классов местами.
* Если train AUC > 0.5 и test AUC < 0.5: в текущем пространстве признаков распределение данных обучения и данных тестирования различно. Менять метки классов опасно.

[Визуализация ROC-AUC](http://www.navan.name/roc/)

**Критерий качества Gini** — отношение площади фигуры, образованной кривой Лоренца и кривой равенства, к площади треугольника, образованного кривыми равенства и неравенства. При оценке качества классификации GINI = 2 × AUC_ROC – 1.

Почему это они так связаны: если в задаче классификации на два класса 0 и 1 интерпретировать эти числа как доходы.

**Площадь под Precision-Recall кривой** — кривая, аналогичная ROC-кривой, только в качестве осей выступают полнота и точность.

<img src="pictures/PR_curve.png" width=400 height=400 />

#### Замечания
**Интерпретация precision и recall**
* Допустим, что мы построили предсказательную модель и получили следущие значения метрик: precision = 0.03, recall = 0.8. О чем нам это говорит? 
* Полнота = 80% => модель все ещё имеет 20% ложных срабатываний;
* Точность = 3% => выявляется только 3% истинно положительных примеров.
* Еще можно объяснить это так:
    * Точность - как много выбранных объектов являются релевантными
    * Полнота - как много релевантных объектов было выбрано

**Неуниверсальность метрики accuracy**
* Accuracy следует использовать только тогда, когда выборка является сбалансированной.
* Пример: у нас есть констанстный классификатор, который классифицирует любой обьект выборки как положительный
    * TP=0
    * TN=100
    * FP=0
    * FN=10
* $accuracy=\frac{0+100}{0+100+0+10}=90.9$
* При этом, наша модель совершенно не обладает никакой предсказательной силой. 

**Проблемы с ROC-AUC**
* ROC-AUC не следует использовать, если положительный класс существенно меньше по размеру.
* ROC-AUC измеряет долю неверно принятых объектов относительно общего числа отрицательных.
* [Неплохая статья, где сравниваются precision + recall и ROC=TPR+FPR](https://towardsdatascience.com/what-metrics-should-we-use-on-imbalanced-data-set-precision-recall-roc-e2e79252aeba)
* Разница заключается в Precision и FPR. Основное различие между этими двумя типами метрик заключается в том, что знаменатель Precision содержит ложные срабатывания, а знаменатель FPR содержит истинные отрицательные значения. Precision измеряет вероятность того, что образец, классифицированный как положительный, на самом деле будет положительным, FPR измеряет соотношение ложноположительных результатов среди отрицательных образцов.
* При большом количестве отрицательных проб precision лучше FPR. Precision больше сосредоточена на положительном классе, чем на отрицательном. Она фактически измеряет вероятность правильного обнаружения положительных значений, в то время как FPR и TPR измеряют способность различать классы.

**Замечание про F-меру**
* F-мера достигает максимума при максимальной полноте и точности, и близка к нулю, если **один из аргументов** близок к нулю. 
* Примере: пусть у нас огромная выборка, в которой один объект имеет метку 1, а все остальные объекты имеют метку 0. При этом у нас константная модель, предсказывающая только класс 1. Тогда:
    * Precision = 0.0
    * Recall = 1.0
* При этом:
    * Арифметическое среднее = 0.5
    * Гармоническое среднее = 0.0

**Цена ошибки**
* В задаче **детектирования правонарушений** на дорогах важна точность, необходимо штрафовать только тех, кто действительно нарушил какие-то правила: precision $\geqslant 0.95$.
* В задаче **обнаружения рака** легких важно найти как можно больше действительно больних пациентов: recall $\geqslant 0.85$.

<img src="pictures/cost_of_error.jpg" width=450 height=450 />

### Мультиклассовая классификация

В случае мультиклассовой классификации матрица ошибок будет выглядеть так:

<img src="pictures/cm_mult.png" width=350 height=350 />

* Микро-усреднение (смещение на частоту класса):
    * Для каждого класса вычисляются TP, FP, FN, показатели суммируются, затем метрики вычисляются для суммированных TP, FP, FN (используется преимущество для несбалансированных выборок);

$$\text{Micro Precision} = \frac{\sum_{i = 1}^{m} TP_i}{\sum_{i = 1}^{m} TP_i + \sum_{i = 1}^{m} FP_i}$$

$$\text{Micro Recall} = \frac{\sum_{i = 1}^{m} TP_i}{\sum_{i = 1}^{m} TP_i + \sum_{i = 1}^{m} FN_i}$$

* Макро-усреднение (принимаем все классы как одиноково важные):
    * Метрики (Precision, Recall, F$_1$) вычисляются независимо для каждого класса, а затем вычисляется среднее арифметическое для них. 
    
$$\text{Macro Precision} = \frac{\sum_{i = 1}^{m} \text{Precision}_i}{m}$$

$$\text{Macro Recall} = \frac{\sum_{i = 1}^{m} \text{Recall}_i}{m}$$

Покажем вычисление метрик для задачи многоклассовой классификации на примере:

Класс 1 - 75 элементов, класс 2 - 50 элементов. Построенный классификатор отнес к первому классу 100 элементов, 50 из которых правильно, и 40 ко второму классу, 30 из которых правильно. Вычислим микро- и макро-характеристики качества классификации (precision, recall, F-score).   

**Макро-характеристики качества классификации:**      
Класс 1: TP = 50, FP = 50, FN = 25     
Точность = TP / (TP + FP) = 50 / (50 + 50) = 1/2     
Полнота = TP / (TP + FN) = 50 / (50 + 25) = 2/3     
F1 = 2 * точность * полнота / (полнота + точность) = 2 * 1/2 * 2/3 / (1/2 + 2/3) = 4/7     

Класс 2: TP = 30, FP = 10, FN = 20     
Точность = TP / (TP + FP) = 30 / (30 + 10) = 3/4     
Полнота = TP / (TP + FN) = 30 / (30 + 20) = 3/5     
F1 = 2 * точность * полнота / (полнота + точность) = 2 * 3/4 * 3/5 / (3/4 + 3/5) = 2/3     

Итоговое F1 = (4/7 + 2/3)/2 = 13/21     

**Микро-характеристики качества классификации:**       
Общие для двух классов TP, FP, FN: TP = 80, FP = 60, FN = 45       
Точность = TP / (TP + FP) = 80 / (80 + 60) = 4/7       
Полнота = TP / (TP + FN) = 80 / (80 + 45) = 16/25       
F1 = 2 * 4/7 * 16/25 / (4/7 + 16/25) = 32/53

## Оценка качества регрессии

В симметричных метриках положительные и отрицательные отклонения рассматриваются в равной степени.

### Симметричные модели

Пусть $X$ - обучающая выборка, $a$ - модель, решающая задачу регрессии, $\overline y = \frac{1}{n} \sum_{i = 1}^{n} y_i$. Тогда:

**Средняя квадратичная ошибка (Mean Squared Error):**
$$MSE = \frac{1}{n} \sum_{i = 1}^{n}(a(x_i) - y_i)^2$$
Средний ответ моделии на выборке. Легко оптимизировать. Чувствителен к выбросам. Среднеквадратичная ошибка подходит для сравнения двух моделей или для контроля качества во время обучения, но не позволяет сделать выводов о том, на сколько хорошо данная модель решает задачу.

MSE применяется в ситуациях, когда нам надо подчеркнуть большие ошибки и выбрать модель, которая дает меньше больших ошибок прогноза. Грубые ошибки становятся заметнее за счет того, что ошибку прогноза мы возводим в квадрат. И модель, которая дает нам меньшее значение среднеквадратической ошибки, можно сказать, что что у этой модели меньше грубых ошибок.

**Cредняя абсолютная ошибка (Mean Absolute Error):**
$$MAE = \frac{1}{n} \sum_{i = 1}^{n}|a(x_i) - y_i|$$
Медиана ответов модели на выборке. Сложнее оптимизировать, так как у модуля не существует производной в нуле.

**Коэффициент детерминации ($0 \leqslant R^2 \leqslant 1$):**
$$R^2 = 1 - \frac{\sum_{i = 1}^{n}(a(x_i) - y_i)^2}{\sum_{i = 1}^{n}(y_i - \overline y)^2}$$
Показывает, какую долю диперсии (разнообразия ответов) во всем целевом векторе модель смогла объяснить. Фактически представляет из себя нормированную среднеквадратичную ошибку. Если значение близко к 1, то модель хорошо объясняет данные, если же к 0, то прогнозы сопоставимы по качеству с константным предсказанием. Метрика крейне чувствительна к выбросам.

**Средняя абсолютная процентная ошибка (Mean Absolute Percentage Error):**
$$MAPE = 100\% × \frac{1}{n} \sum_{i = 1}^{n} \frac{|y_i − a(x_i)|}{|yi|}$$
Показывает процент ошибок модели. Не имеет размерности. Основная проблема данной метрики — нестабильность.

**Корень из средней квадратичной ошибки (Root Mean Squared Error):**
$$RMSE = \sqrt{\frac{1}{n} \sum_{i = 1}^{n} (y_i − \overline y_i)^2}$$
Также существует проблема нестабильности.

**Cимметричная MAPE (Symmetric MAPE):**
$$SMAPE = \frac{1}{n} \sum_{i = 1}^{n} \frac{2 × |y_i − a(x_i)|}{|yi| + |a(x_i)|}$$

**Средняя абсолютная масштабированная ошибка (Mean absolute scaled error):**
$$MASE = \frac{\sum_{i = 1}^{n} |Y_i − e_i|}{\frac{n}{n - 1} \sum_{i = 2}^{n} |Y_i − Y_{i−1}|}$$
Метрика не зависит от масштабов данных и предоставляет несмещенную оценку. Сумма в числителе соответствует тестовой выборке, в знаменателе - обучающей. Данную метрику тяжело интерпретировать. 

### Несимметричные модели

**Квантильная ошибка ($0 \leqslant \tau \leqslant 1$):**
$$\rho_\tau = \frac{1}{n} \sum_{i = 1}^{n} ((\tau - 1)[y_i < a(x_i)] + \tau[y_i \geqslant a(x_i)])(y_i - a(x_i))$$
Определяет то, за что нужно штрафовать сильнее: за недообучение или переобучение.

<img src="pictures/kvantilfp.png" width=350 height=350 />

### Замечания
MSE более чувствителен к выбросам, чем MAE. Необходимо оценить объекты в выборке для выбора лучшей метрики.

## Оценка качества кластеризации
### Внешняя оценка качества
* A — число пар элементов, находящихся в одном кластере как в полученном, так и в эталонном разбиении (like TP)
* B — число пар элементов, находящихся в разных кластерах как в полученном, так и в эталонном разбиении (like TN)
* C — число пар элементов, находящихся в одном кластере полученного, но в разных эталонного разбиения (like FP)
* D — число пар элементов, находящихся в разных кластерах полученного, но в одном эталонного разбиения (like FN)

Меры:
* Rand Measure: $RandIndex = \frac{A+B}{A+B+C+D}$
* F-measure: $Pr = \frac{A}{A+C}$, $Re = \frac{A}{A+D}$, $ F_1 = \frac{2PrRe}{Pr+Re}$
* Индекс Жакара: $J = \frac{A}{A+C+D}$
* Чистота (purinity) - отношение между доминантным классом в кластере и размером кластера $\omega_i$: $purinity(\omega_i) = \frac{1}{n_i} \max\limits_{j} n_{ij}, j \in C$

**Пример вычисления чистоты:**     
* Cluster I: $Purity = \frac{1}{6} (max(5, 1, 0)) = \frac{5}{6}$
* Cluster II: $Purity = \frac{1}{6} (max(1, 4, 0)) = \frac{4}{6}$
* Cluster III: $Purity = \frac{1}{6} (max(2, 0, 3)) = \frac{3}{5}$

<img src="pictures/purinity.png" width=380 height=380 />

### Внутренняя оценка качества
* среднее внутрикластерное расстояние должно быть как можно меньше:
$$F_0 = \frac{\sum\limits_{i<j} [y_i = y_j] \rho(x_i, x_j)}{\sum\limits_{i<j} [y_i = y_j]} \rightarrow \mbox{min}$$
* среднее межкластерное расстояние должно быть как можно больше:
$$F_1 = \frac{\sum\limits_{i<j} [y_i \neq y_j] \rho(x_i, x_j)}{\sum\limits_{i<j} [y_i \neq y_j]} \rightarrow \mbox{max}$$

## Литература
1. [www.coursera.org - Специализация Машинное обучение и анализ данных](https://www.coursera.org/specializations/machine-learning-data-analysis)
2. [neerc.ifmo.ru - Оценка качества в задачах классификации и регрессии](https://neerc.ifmo.ru/wiki/index.php?title=%D0%9E%D1%86%D0%B5%D0%BD%D0%BA%D0%B0_%D0%BA%D0%B0%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%B0_%D0%B2_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87%D0%B0%D1%85_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D0%B8_%D0%B8_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D0%B8)
3. [neerc.ifmo.ru - Оценка качества в задачах классификации](http://neerc.ifmo.ru/wiki/index.php?title=%D0%9E%D1%86%D0%B5%D0%BD%D0%BA%D0%B0_%D0%BA%D0%B0%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%B0_%D0%B2_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87%D0%B0%D1%85_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D0%B8)
4. [arxiv-vanity.com - Metrics for Multi-Class Classification: an Overview](https://www.arxiv-vanity.com/papers/2008.05756/)
5. [wikipedia.org - F-score](https://en.wikipedia.org/wiki/F-score)
6. [AUC ROC (площадь под кривой ошибок)](https://dyakonov.org/2017/07/28/auc-roc-%D0%BF%D0%BB%D0%BE%D1%89%D0%B0%D0%B4%D1%8C-%D0%BF%D0%BE%D0%B4-%D0%BA%D1%80%D0%B8%D0%B2%D0%BE%D0%B9-%D0%BE%D1%88%D0%B8%D0%B1%D0%BE%D0%BA/)